<a href="https://colab.research.google.com/github/xquiroz/Analisis-de-datos/blob/main/Clasificaci%C3%B3n_Netflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Revisión de los datos


In [ ]:
#Importar librerías básicas
import pandas as pd
import numpy as np
import time as t

# Cargar el conjunto de datos de la plataforma
dataset = pd.read_csv('Netflix Userbase.csv')
#Limpiamos el dataset con las columnas inecesarias
dataset = dataset.drop('User ID',axis=1)
dataset = dataset.drop('Monthly Revenue',axis=1)
dataset = dataset.drop('Plan Duration',axis=1)
dataset = dataset.drop('Join Date',axis=1)
#al dia de Hoy 29/07/23 que cuentas han sido cancelada, que no han hecho pago
#Modificamos la columna de 'Last Payment Date'
Lastpay = dataset['Last Payment Date']
Lastpay
#Usamos la fecha un mes antes para saber quienes han cancelado
cancel = "29/06/2023"
cancel = t.strptime(cancel, "%d/%m/%Y")
#Mediante un for vemos las fechas de cada uno de los datos para saber
#quienes han cancelado comparandolo con su ultimo dia de pago
for i in range(2500):
  Lastpay[i]= t.strptime(Lastpay[i], "%d/%m/%Y")
  if cancel > Lastpay[i]: # si la fecha es antes del 29/06/2023
    Lastpay[i]= 1 #Cancelo
  else:
    Lastpay[i]= 0 #no cancelo
#renombramos la columna que acabamos de modificar
dataset = dataset.rename( columns={'Last Payment Date':'Cancel'})
dataset



<ipython-input-27-2dead2d85551>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Lastpay[i]= t.strptime(Lastpay[i], "%d/%m/%Y")
<ipython-input-27-2dead2d85551>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Lastpay[i]= 1 #Cancelo
<ipython-input-27-2dead2d85551>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Lastpay[i]= 0 #no cancelo


,Subscription Type,Cancel,Country,Age,Gender,Device,Suscription_duration
0,Basic,1,United States,28,Male,Smartphone,17
1,Premium,1,Canada,35,Female,Tablet,21
2,Standard,1,United Kingdom,42,Male,Smart TV,4
3,Standard,1,Australia,51,Female,Laptop,11
4,Basic,1,Germany,33,Male,Smartphone,1
...,...,...,...,...,...,...,...
2495,Premium,0,Spain,28,Female,Smart TV,12
2496,Basic,0,Spain,33,Female,Smart TV,11
2497,Standard,0,United States,38,Male,Laptop,11
2498,Standard,0,Canada,48,Female,Tablet,11


In [ ]:
# Remplazamos los datos a enteros
#Tipo de suscripcion
dataset = dataset.replace('Basic', 1, regex= True)
dataset = dataset.replace('Standard', 2, regex= True)
dataset = dataset.replace('Premium', 3, regex= True)
#Pais de origen
dataset = dataset.replace('Australia', 1, regex= True)
dataset = dataset.replace('Brazil', 2, regex= True)
dataset = dataset.replace('Canada', 3, regex= True)
dataset = dataset.replace('France', 4, regex= True)
dataset = dataset.replace('Germany', 5, regex= True)
dataset = dataset.replace('Italy', 6, regex= True)
dataset = dataset.replace('Mexico', 7, regex= True)
dataset = dataset.replace('Spain', 8, regex= True)
dataset = dataset.replace('United Kingdom', 9, regex= True)
dataset = dataset.replace('United State', 10, regex= True)
#Sexo
dataset = dataset.replace('Male', 0, regex= True)
dataset = dataset.replace('Female', 1, regex= True)
#Dispositivo
dataset = dataset.replace('Laptop', 1, regex= True)
dataset = dataset.replace('Smart TV', 2, regex= True)
dataset = dataset.replace('Smartphone', 3, regex= True)
dataset = dataset.replace('Tablet', 4, regex= True)
#Vemos el tipo de datos
dataset
print(dataset.dtypes)

Subscription Type       int64
Cancel                  int64
Country                 int64
Age                     int64
Gender                  int64
Device                  int64
Suscription_duration    int64
dtype: object


In [ ]:
dataset

,Subscription Type,Cancel,Country,Age,Gender,Device,Suscription_duration
0,1,1,10,28,0,3,17
1,3,1,3,35,1,4,21
2,2,1,9,42,0,2,4
3,2,1,1,51,1,1,11
4,1,1,5,33,0,3,1
...,...,...,...,...,...,...,...
2495,3,0,8,28,1,2,12
2496,1,0,8,33,1,2,11
2497,2,0,10,38,0,1,11
2498,2,0,3,48,1,4,11


Proceso de clasificación

In [ ]:
# Importar las bibliotecas necesarias
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X = dataset.drop('Cancel', axis=1)#independiente
y = dataset['Cancel'] #dependiente
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un clasificador de Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100)

# Entrenar el clasificador en el conjunto de entrenamiento
rf_classifier.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = rf_classifier.predict(X_test)

# Calcular la precisión de las predicciones
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy*100:.2f}%')


Precisión del modelo: 77.80%


Matriz de confusión

In [ ]:
cm  =confusion_matrix(y_test, y_pred)
print(cm)

[[909  41]
 [211  89]]


Predicciones


In [ ]:
# Crear nuevo conjunto de datos para hacer una predicción
conjunto1 = [[1,5,33,0,3,1]]

# Realizar una predicción del conjunto de datos
predicted_class = rf_classifier.predict(conjunto1)

# El resultado es la clase predicha para el conjunto de datos
print(f'Clase predicha para el conjunto: {predicted_class[0]}')

Clase predicha para el conjunto: 1


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
 #Crear nuevo conjunto de datos para hacer una predicción
conjunto2 = [[1,5,33,0,3,12]]

# Realizar una predicción del conjunto de datos
predicted_class = rf_classifier.predict(conjunto2)

# El resultado es la clase predicha para el conjunto de datos
print(f'Clase predicha para el conjunto2: {predicted_class[0]}')

Clase predicha para el conjunto2: 0


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
